# MYSQL PRACTICE

In [ ]:
!pip install pymysql > /dev/null

In [ ]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

In [ ]:
import json
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)

In [ ]:
import pymysql
conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port']
)

## PASSWORD 암호화

### Secure Hash Algorithm(SHA-256)

In [8]:
import hashlib
pwd = '1234'
pwd_sha256 = hashlib.sha256(pwd.encode())
pwd_sha256.digest()

b'\x03\xacgB\x16\xf3\xe1\\v\x1e\xe1\xa5\xe2U\xf0g\x956#\xc8\xb3\x88\xb4E\x9e\x13\xf9x\xd7\xc8F\xf4'

### Base64

In [9]:
import base64
base64.b64encode(pwd_sha256.digest())

b'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

### PASSWORD 생성 함수

In [10]:
import hashlib
import base64

def gen_pwd(pwd):
    pwd_sha256 = hashlib.sha256(pwd.encode())
    hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('UTF-8')
    return hashed_pwd

In [6]:
gen_pwd('qwerty')

'ZehL4zUy+3hMSBKWdfnv86aCsnFowOp0Syz1juAjN8U='

## users 테이블에 PASSWORD 입력

### uid = 21

In [ ]:
sql = 'SELECT uid FROM users;'
cur = conn.cursor()
cur.execute(sql)
result = cur.fetchall()
result

In [ ]:
sql = 'UPDATE users SET pwd=%s WHERE uid=%s;'
for line in results:
    uid = line[0]
    pwd = gen_pwd(uid='21')
    cur.execute(sql, (pwd, uid))

In [ ]:
conn.commit()

### email 추가 : uid@mc.com

In [ ]:
sql_email = 'UPDATE users SET email=%s WHERE uid=%s;'
for line in results:
    uid = line[0]
    email = uid + '@mc.com'
    cur.execute(sql, (email, uid))
conn.commit()

## 로그인

In [ ]:
sql = 'SELECT uid, pwd FROM users WHERE uid=%s AND is_deleted=0;'

### CASE 1 : uid가 없는 경우

In [ ]:
uid = 'fool'
cur.execute(sql, (uid,))
result = cur.fetchone()
result

### CASE 2 : PASSWORD가 다른 경우

In [ ]:
uid = 'ABC'
pwd = 'asdf'
cur.execute(sql, (uid,))
result = cur.fetchone()
result

In [ ]:
db_pwd = result[1]
hashed_pwd = gen_pwd(pwd)
if db_pwd == hashed_pwd:
    print('LOGIN')
else:
    print('INCORRECT PASSWORD')

### CASE 3 : OK

In [ ]:
uid = 'ABC'
pwd = '1234'
cur.execute(sql, (uid,))
result = cur.fetchone()
db_pwd = result[1]
hashed_pwd = gen_pwd(pwd)
if db_pwd == hashed_pwd:
    print('LOGIN')
else:
    print('INCORRECT PASSWORD')

### 통합 코드

In [ ]:
sql = 'SELECT uid, pwd FROM users WHERE uid=%s AND is_deleted=0;'
uid = 'ABC'
pwd = '1234'
cur.execute(sql, (uid,))
result = cur.fetchone()
if result:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('LOGIN')
    else:
        print('INCORRECT PASSWORD')
else:
    print('INCORRECT ID')

### 예외처리 이용

In [ ]:
sql = 'SELECT uid, pwd FROM users WHERE uid=%s AND is_deleted=0;'
uid = 'ABC'
pwd = '1234'
cur.execute(sql, (uid,))
result = cur.fetchone()
try:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('LOGIN')
    else:
        print('INCORRECT PASSWORD')
except:
    print('INCORRECT ID')

## 종료

In [ ]:
cur.close()
conn.close()